# **3. LiDAR Point clouds to 3D surfaces**

In this final tutorial, we'll see how PyGMT can be used
to perform more advanced geoprocessing workflows
such as filtering and interpolation of point datasets to grids.
At the end, we'll also experiment with making some 3D perspective plots
of a Digital Surface Model (DSM) we produced through this exercise!

## **Getting started**

Let's start by importing the Python libraries we'll be using.
We'll be using [laspy](https://github.com/laspy/laspy) to read in LAZ LiDAR files,
and [rioxarray](https://corteva.github.io/rioxarray) to export our DSM to a GeoTIFF later on.
Hopefully [pandas](https://pandas.pydata.org) and [pygmt](https://www.pygmt.org)
will be familiar to you by now.

In [ ]:
import laspy
import pandas as pd
import pygmt
import rioxarray

## **3.1 Loading a LiDAR point cloud**

This time we'll be visiting [Picton](https://en.wikipedia.org/wiki/Picton%2C_New_Zealand)
which is a town 65 km west of New Zealand's capital Wellington.
They've had a fairly recent LiDAR survey done between May and September 2018,
with the point cloud and derived products published at 
[OpenTopography](https://doi.org/10.5069/G91R6NNG) and
[LINZ Data Service](https://data.linz.govt.nz/layer/103538-marlborough-lidar-index-tiles-2018/)
(under a [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/) license).
This tutorial is actually inspired by this
[blog post](https://medium.com/on-location/from-points-to-pixels-creating-digital-elevation-models-from-open-topography-point-clouds-abe616d06860)
published by LINZ.

First though, we'll need to download a sample file to play with.
Luckily for us,
there is a function called [pygmt.which](https://www.pygmt.org/dev/api/generated/pygmt.which)
that has to ability to do just that!
The download="l" option can be used to download a remote file to our local working directory.

In [ ]:
lazfile = pygmt.which(
    fname="https://cloud.sdsc.edu/v1/AUTH_opentopography/PC_Bulk/NZ18_Marlbo/CL2_BQ29_2018_1000_1607.laz",
    download="l"
)
print(lazfile)

Now we can use [laspy](https://github.com/laspy/laspy) to read in our example LAZ file
into a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).
The data will be kept in a variable called 'df' which stands for dataframe.

In [ ]:
with laspy.file.File(lazfile, mode="r") as lasdata:
    df = pd.DataFrame(data=lasdata.points["point"])
    df.X = lasdata.get_x_scaled()
    df.Y = lasdata.get_y_scaled()
    df.Z = lasdata.get_z_scaled()

### **Quick data preview**

Let's see how many data points (rows) we have,
get the bounding box region of our study area,
and also take a look at a snippet of our table.

In [ ]:
region = [df.X.min(), df.X.max(), df.Y.min(), df.Y.max()]
print(f"Total of {len(df)} data points, covering region {region}")
df.head()

More than 2 million points, that's a lot!
Let's try to take a quick look of our LiDAR elevation points on a map.
PyGMT *can* plot these many points, but it might take a long time,
so let's do a quick filter by taking every
[n-th row](https://stackoverflow.com/questions/25055712/pandas-every-nth-rowhttps://stackoverflow.com/questions/25055712/pandas-every-nth-row)
from our dataframe table.

In [ ]:
df_filtered = df.iloc[::20]
print(f"Filtered down to {len(df_filtered)} data points")

Now we can visualize these quickly on a map with PyGMT.

In [ ]:
fig = pygmt.Figure()
fig.basemap(frame=True, region=region, projection="x1:5000")
pygmt.makecpt(cmap="earth", series=[df.Z.min(), df.Z.max()])
fig.plot(x=df_filtered.X, y=df_filtered.Y, color=df_filtered.Z, style="c0.05c", cmap=True)
fig.colorbar(position="JMR", frame=["af", 'y+l"Elevation (m)"'], X="1.5c")
fig.show()

It's hard to make out what the objects are,
but hopefully you'll see what looks like a wharf with docks on the top left corner.
This is basically showing us a part of the port of Picton.

## **3.2 Creating a Digital Surface Model (DSM)**

Let's now produce a digital surface model from our point cloud.
We'll first do some proper spatial data cleaning using both
[pandas](https://pandas.pydata.org) and [pygmt](https://www.pygmt.org).

The first step is to remove the LiDAR points classified as water
(see [ASPRS LAS Specification](https://www.asprs.org/a/society/committees/standards/LAS_1_4_r13.pdf)),
which is done by querying all the points that are not '9'.

In [ ]:
df = df.query(expr="raw_classification != 9")

Next, we'll use the [blockmedian](https://pygmt-git-foss4g2019oceania.gmt.now.sh/api/generated/pygmt.blockmedian.html#pygmt.blockmedian)
to trim the points down spatially.
By default, this function will compute a single median elevation for every pixel on an equally spaced grid.

For a DSM though,
we should technically be picking the highest elevation instead of the median.
We'll tell blockmedian to use the 99th [quantile](https://docs.generic-mapping-tools.org/latest/blockmedian.html#t) (T) instead,
and set our output grid spacing to be exactly 1 m (1+e).
Note that we'll only be taking the X, Y, and Z (elevation) columns for this.

In [ ]:
df_trimmed = pygmt.blockmedian(table=df[["X", "Y", "Z"]], T=0.99, spacing="1+e", region=region)
df_trimmed

Lastly, we'll use [pygmt.surface](https://www.pygmt.org/dev/api/generated/pygmt.surface)
to produce a grid from the x, y, z data points.
We'll make sure to set our desired grid spacing to be exactly 1 m (1+e) again,
and also set a tension factor (T) of 0.35 which is suitable for steep topography
(since we have some buildings with 'steep' vertical edges).
Note that this might take some time to run.

In [ ]:
grid = pygmt.surface(
    x=df_trimmed.X,
    y=df_trimmed.Y,
    z=df_trimmed.Z,
    spacing="1+e",
    region=region,
    T=0.35
)

Let's take a look at the output.
The grid will be returned as an
[xarray.Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html#xarray.Dataset),
with each pixel's elevation stored in the data variable 'z'.

In [ ]:
grid

To look at the actual elevation values,
we need to call `grid.z` which gives us an
[xarray.DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html#xarray.DataArray)
(i.e. an xarray.DataArray is one variable inside of an xarray.Dataset).

In [ ]:
grid.z

## **3.3 Plotting a Digital Surface Model (DSM)**

### **Plotting in 2D**

Now we can plot our Digital Surface Model (DSM) grid!
Since [grdimage](https://www.pygmt.org/dev/api/generated/pygmt.Figure.grdimage)
requires our grid to be an xarray.DataArray instead of an xarray.Dataset,
we'll need to pass in 'grid.z' instead of just 'grid'.

In [ ]:
fig2 = pygmt.Figure()
fig2.basemap(frame=True, region=region, projection="x1:5000")
pygmt.makecpt(cmap="earth", series=[-10, 50])
fig2.grdimage(grid=grid.z)
fig2.colorbar(position="JMR", frame=["af", 'y+l"Elevation (m)"'], X="1.2c")
fig2.show()

### **Plotting a 3D perspective view!**

Now comes the cool part.
PyGMT has a [grdview](https://pygmt-git-foss4g2019oceania.gmt.now.sh/api/generated/pygmt.Figure.grdview.html#pygmt.Figure.grdview)
function that has the ability to make high quality 3D perspective plots of our elevation surface!

Besides providing the elevation 'reliefgrid', there are a few basic things to configure:
- cmap - name of colormap to use
- surftype - we'll use 's' here which just creates a regular surface
- perspective - set azimuth angle (e.g. 45 for NorthEast) and elevation (e.g 30 degrees) of the viewpoint
- zscale - a vertical exaggeration factor, we'll use 0.1 here.

In [ ]:
fig3 = pygmt.Figure()
fig3.grdview(
    reliefgrid=grid.z,
    cmap="earth",
    surftype="s",  # surface plot
    perspective=[45, 30],  # azimuth bearing, and elevation angle
    zscale=0.1,  # vertical exaggeration
)
fig3.show()

There are also other things we can add such as a proper 3D map frame with:
- automatic tick marks on x and y axis (e.g. xaf+l"Label")
- z-axis automatic tick marks (zaf+l"Label")
- plot title (+t"Title")

In [ ]:
fig3 = pygmt.Figure()
fig3.grdview(
    reliefgrid=grid.z,
    cmap="earth",
    surftype="s",  # surface plot
    perspective=[45, 30],  # azimuth bearing, and elevation angle
    zscale=0.1,  # vertical exaggeration
    frame=['xaf+l"Easting"', 'yaf+l"Northing"', 'zaf+l"Elevation (m)"', '+t"Picton Harbour"']
)
fig3.show()

Before moving on,
have a go at changing the azimuth and elevation angle to get a different view.
Try to tweak the vertical exaggeration factor and play around with the map frame.

### **Save the figure and output DSM to a GeoTIFF**

Remember to save your work!
We'll save each of our figures into separate files first.

In [ ]:
fig.savefig(fname="picton_1d_lidar.png")
fig2.savefig(fname="picton_2d_dsm_map.png")
fig3.savefig(fname="picton_3d_dsm_view.png")

Also, it'll be good to have a proper GeoTIFF of the DSM we made.
This can be done using
[rioxarray's to_raster method](https://corteva.github.io/rioxarray/html/examples/convert_to_raster.html).

In [ ]:
grid.z.rio.to_raster(raster_path="DSM_of_Picton.tif")

The files should now appear on the file list on the left,
and you can download them to your computer.

## **3.4 Exploration time - Create a 3D surface map of another area**

Yet another challenge to test your PyGMT skills!
You'll be processing a LiDAR point cloud of a different area from start to finish.
Good luck!

Feel free to find a dataset from an area you're interested in using
[OpenTopography](http://opentopo.sdsc.edu/datasets).
To make it easier though, I've provided an example for [Queenstown, New Zealand](https://data.linz.govt.nz/layer/99121-otago-queenstown-lidar-index-tiles-2016/) below:
  (OpenTopography [link](http://opentopo.sdsc.edu/lidarDataset?opentopoID=OTLAS.012019.2193.1)):
  - https://cloud.sdsc.edu/v1/AUTH_opentopography/PC_Bulk/NZ16_Queen/CL2_CC11_2016_1000_0813.laz
  - https://cloud.sdsc.edu/v1/AUTH_opentopography/PC_Bulk/NZ16_Queen/CL2_CC11_2016_1000_0814.laz
  - https://cloud.sdsc.edu/v1/AUTH_opentopography/PC_Bulk/NZ16_Queen/CL2_CC11_2016_1000_0913.laz
  - https://cloud.sdsc.edu/v1/AUTH_opentopography/PC_Bulk/NZ16_Queen/CL2_CC11_2016_1000_0914.laz

### **Download and load data**

In [ ]:
filenames = [
    "",
]

df = pd.DataFrame()  # Start an empty DataFrame
for fname in filenames:
    lazfile = pygmt.which(
        fname=fname,
        download="l"
    )
    with laspy.file.File(lazfile, mode="r") as lasdata:
        temp_df = pd.DataFrame(data=lasdata.points["point"])
        temp_df.X = lasdata.get_x_scaled()
        temp_df.Y = lasdata.get_y_scaled()
        temp_df.Z = lasdata.get_z_scaled()
    df = df.append(temp_df, ignore_index=True)  # load each point cloud into the DataFrame

In [ ]:
df

### **Create a DSM**

### **Plot the DSM**

That's it!
We should have some spare time to cover any lingering questions in more depth.
If any of you have any questions afterwards the workshop/conference,
feel free to post them on our [community forum](https://forum.generic-mapping-tools.org)
and we'll be more than happy to assist.